In [ ]:
# Install required libraries
!pip install -q langchain openai crewai python-dotenv

# Imports
import os
import logging
import json
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from crewai import Agent, Task, Crew, Process

In [ ]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s'
)

# Load environment variables (ensure you have a .env file)
load_dotenv()

# Set OpenAI API Key (replace with your actual key)
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# Sample Lead Data
lead_data = {
    "name": "Emily Rodriguez",
    "job_title": "Marketing Director",
    "company": "TechInnovate Solutions",
    "email": "emily.rodriguez@techinnovate.com",
    "linkedin": "linkedin.com/in/emilyrodriguez",
    "interests": ["AI marketing", "digital transformation", "startup growth"]
}

# Product Details
product_details = {
    "name": "AIReach",
    "tagline": "AI-Powered Sales Engagement Platform",
    "key_features": [
        "Personalized Email Generation",
        "LinkedIn Profile Scraping",
        "Advanced Lead Scoring"
    ],
    "unique_value_prop": "Revolutionize your sales outreach with AI-driven personalization"
}

In [ ]:
# Initialize Language Model
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,
    max_retries=3
)

# Research Agent
research_agent = Agent(
    role="Lead Research Specialist",
    goal="Deep dive into lead's professional context and identify personalization opportunities",
    backstory="""
    You are an expert in analyzing professional profiles,
    understanding nuanced business contexts, and identifying
    precise personalization strategies for sales outreach.
    """,
    verbose=True,
    llm=llm,
    max_iterations=3
)

# Email Crafting Agent
email_agent = Agent(
    role="Sales Engagement Copywriter",
    goal="Craft compelling, hyper-personalized email communication",
    backstory="""
    A masterful communicator who transforms raw insights
    into persuasive, tailored email content that resonates
    with high-level professionals and sparks genuine interest.
    """,
    verbose=True,
    llm=llm,
    max_iterations=3
)

In [ ]:
# Research Task
research_task = Task(
    description=f"""
    Comprehensive Lead Profile Analysis for {lead_data['name']}:

    Professional Context:
    - Detailed examination of job role and company
    - Identify potential business challenges
    - Map alignment with AIReach's capabilities

    Specific Objectives:
    1. Extract 3-4 unique personalization insights
    2. Understand professional pain points
    3. Correlate lead's interests with product value
    4. Prepare contextualized communication strategy
    """,
    agent=research_agent,
    expected_output="Detailed personalization insights and communication strategy"
)

# Email Generation Task
email_task = Task(
    description=f"""
    Personalized Email Composition for {lead_data['name']}:

    Utilizing Research Insights:
    - Craft a subject line that captures attention
    - Develop a conversational yet professional email body
    - Highlight AIReach's unique value proposition
    - Create a subtle, effective call-to-action

    Communication Guidelines:
    1. Use insights from research task
    2. Demonstrate clear understanding of lead's context
    3. Showcase product relevance
    4. Maintain professional, engaging tone
    """,
    agent=email_agent,
    expected_output="Complete email draft with personalized subject and body"
)

In [ ]:
# Create Crew for Email Generation
email_generation_crew = Crew(
    agents=[research_agent, email_agent],
    tasks=[research_task, email_task],
    verbose=2,
    process=Process.sequential,
    memory=True
)

# Execute Email Generation
try:
    # Kickoff the crew and generate email
    generated_email = email_generation_crew.kickoff()

    # Log the result
    logging.info("Email Generation Successful")
    print("\n--- Generated Email ---")
    print(generated_email)

except Exception as e:
    logging.error(f"Email Generation Failed: {str(e)}")
    print("Error in email generation:", str(e))

In [ ]:
def validate_email_output(email_content, lead_data, product_details):
    """
    Validate generated email content
    """
    validation_results = {
        "personalization_score": 0,
        "is_valid": True,
        "feedback": []
    }

    # Check personalization elements
    personalization_checks = [
        lead_data['name'],
        lead_data['company'],
        lead_data['job_title']
    ]

    for check in personalization_checks:
        if check.lower() in email_content.lower():
            validation_results['personalization_score'] += 1
        else:
            validation_results['feedback'].append(f"Missing personalization: {check}")

    # Check product mentions
    product_checks = [
        product_details['name'],
        product_details['tagline']
    ]

    for check in product_checks:
        if check.lower() in email_content.lower():
            validation_results['personalization_score'] += 1
        else:
            validation_results['feedback'].append(f"Missing product reference: {check}")

    # Overall validation
    validation_results['is_valid'] = validation_results['personalization_score'] >= 3

    return validation_results

# Validate the generated email
if 'generated_email' in locals():
    validation_result = validate_email_output(generated_email, lead_data, product_details)

    print("\n--- Email Validation Results ---")
    print(f"Personalization Score: {validation_result['personalization_score']}/4")
    print(f"Email Valid: {validation_result['is_valid']}")

    if validation_result['feedback']:
        print("Improvement Suggestions:")
        for feedback in validation_result['feedback']:
            print(f"- {feedback}")

In [ ]:
--- Generated Email ---
Subject: Transforming Marketing Strategies at TechInnovate with AIReach

Hi Emily,

I hope this message finds you well. As a Marketing Director at TechInnovate Solutions,
I understand the challenges of driving digital transformation...

[Personalized email body highlighting AIReach's capabilities]

--- Email Validation Results ---
Personalization Score: 4/4
Email Valid: True

In [ ]:
# Create Dockerfile
dockerfile_content = """
# Use an official Python runtime as a parent image
FROM python:3.9-slim

# Set the working directory in the container
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    build-essential \
    curl \
    software-properties-common \
    git \
    && rm -rf /var/lib/apt/lists/*

# Copy the current directory contents into the container at /app
COPY . /app

# Install any needed packages specified in requirements.txt
COPY requirements.txt /app/
RUN pip install --no-cache-dir -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME AIReachEmailGenerator

# Run the script when the container launches
CMD ["python", "email_generator.py"]
"""

# Write Dockerfile
with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)

# Create requirements.txt
requirements_content = """
# Core Libraries
langchain==0.1.0
openai==1.0.0
crewai==0.5.0
python-dotenv==1.0.0

# Additional Dependencies
requests==2.28.2
pydantic==2.0.0
typing-extensions==4.5.0
"""

# Write requirements.txt
with open('requirements.txt', 'w') as f:
    f.write(requirements_content)

In [ ]:
# Create docker-compose.yml
docker_compose_content = """
version: '3.8'

services:
  email-generator:
    build: .
    container_name: aireach-email-generator
    environment:
      - OPENAI_API_KEY=${OPENAI_API_KEY}
    volumes:
      - .:/app
    restart: unless-stopped
"""

# Write docker-compose.yml
with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

In [ ]:
# email_generator.py
import os
import logging
from dotenv import load_dotenv

# Import previous modules
from email_generation_workflow import generate_email  # Assume previous code is in this module

def main():
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s: %(message)s',
        filename='email_generator.log'
    )

    try:
        # Load environment variables
        load_dotenv()

        # Sample lead and product data
        lead_data = {
            "name": "Emily Rodriguez",
            "job_title": "Marketing Director",
            "company": "TechInnovate Solutions",
            "email": "emily.rodriguez@techinnovate.com"
        }

        product_details = {
            "name": "AIReach",
            "tagline": "AI-Powered Sales Engagement Platform"
        }

        # Generate email
        generated_email = generate_email(lead_data, product_details)

        # Log successful generation
        logging.info("Email generated successfully")
        print(generated_email)

    except Exception as e:
        logging.error(f"Error in email generation: {str(e)}")
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
# Shell script for easy Docker management
docker_build_script = """
#!/bin/bash

# Build Docker image
docker build -t aireach-email-generator .

# Run Docker container
docker run -e OPENAI_API_KEY=$OPENAI_API_KEY aireach-email-generator
"""

# Write build script
with open('build_and_run.sh', 'w') as f:
    f.write(docker_build_script)

# Make script executable
!chmod +x build_and_run.sh

In [ ]:
# Create GitHub Actions workflow
github_actions_content = """
name: AIReach Email Generator CI

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build:
    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v2

    - name: Set up Python
      uses: actions/setup-python@v2
      with:
        python-version: '3.9'

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Build Docker Image
      run: docker build -t aireach-email-generator .

    - name: Run Tests
      run: python -m unittest discover tests
"""

# Write GitHub Actions workflow
with open('.github/workflows/ci.yml', 'w') as f:
    f.write(github_actions_content)

In [ ]:
# Build Docker image
docker build -t aireach-email-generator .

# Run Docker container
docker run -e OPENAI_API_KEY=your_key aireach-email-generator

# Using Docker Compose
docker-compose up --build